# Searching Density Bursting Subgraphs in the Facebook Wall Posts dataset

## Necessary imports

In [ ]:
# import external libraries
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# import custom classes and functions
from src import Graph, TemporalGraph, find_density_bursting_subgraph

## Data preparation

First off, load the data.

In [ ]:
with open("data/sample.txt", "r") as f:
    s = f.read()
connections = np.array([x.split(" ") for x in s.split("\n")], dtype=int)

Look at the dataset and gather some information about it.

In [ ]:
# top rows
df = pd.DataFrame(connections, columns=['facebook_user_1', 'facebook_user_1', 'day'])
df.head(10)

We show some statistics, although here, they don't really make sense.

In [ ]:
df.describe()

We grab a sample. More precisely, we select the bottom rows of the dataset  which correspond to the most ancient wall posts in ascending order.

In [ ]:
max_timestep = 5
connections = connections[np.where(connections[:, 2] < max_timestep)]

In [ ]:
users = np.unique(connections[:, :2])
n_users = users.shape[-1]
new_nodes = dict(np.vstack((users, np.arange(n_users))).T)

The row data consists of edge lists (one for each timestep). We transform it into a temporal graph.

In [ ]:
snapshots = []
for t in range(max_timestep):
    adj_mat = np.zeros((n_users, n_users))
    for x in connections[np.where(connections[:, 2] == t), :2][0]:
        i, j = x
        adj_mat[new_nodes[i], new_nodes[j]] += 1
        adj_mat[new_nodes[j], new_nodes[i]] += 1
    snapshots += [Graph(list(new_nodes.values()), adj_mat)]

In [ ]:
t_graph = TemporalGraph(snapshots, verbose=True)

## Graph visualization

Once we have a temporal graph, we can visualize its snapshots and accumulated graphs. We can play with it by changing the value of t and re-running the three followig cells.

In [ ]:
# select a timestep
t = 5

if True:
    # to plot the accumulated graph
    graph = np.sum(t_graph[:t+1])
else:
    # to plot the snapshot
    graph = np.sum(t_graph[t])

In [ ]:
edges = [(i, j, {
    "weight": int(graph.adjacency_matrix[i, j])
}) for i in range(graph.n_vertices) for j in range(i)
         if graph.adjacency_matrix[i, j]]

net = nx.Graph()
net.add_edges_from(edges)

In [ ]:
pos = nx.spring_layout(net)
nx.draw(net, pos, with_labels=True)
plt.show()

## Running the algorithm

We now call the _find_density_bursting_subgraph_ function which takes as inputs the temporal graph from and a positive integer theta which is the minimum duration threshold (the minimum amount above of timesteps for a density bursting graph). We obtain a subset of nodes and a time interval which, together, define a temporal subgraph which we believe to be a density bursting subgraph.

In [ ]:
nodes, t_b, t_e = find_density_bursting_subgraph(t_graph, theta=5)

In [ ]:
print(f"nodes : {nodes}\nT = ({t_b}, {t_e}]")